In [1]:
# install any necessary packages
# !pip install -U spacy
# !pip install spacy_transformers

# Install below packages if error occurs while installing above packages
#!pip install -U pip setuptools
# !pip install typing-extensions==4.6.1


https://medium.com/@mjghadge9007/building-your-own-custom-named-entity-recognition-ner-model-with-spacy-v3-a-step-by-step-guide-15c7dcb1c416

In [ ]:
# Import the necessary libraries and mount Google Drive to access project files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change the working directory to the project folder
%cd "/content/drive/MyDrive/01_PoC/02_projects/00_NER"

/content/drive/MyDrive/01_PoC/02_projects/00_NER


In [3]:
# Import required libraries and install any necessary packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

# Check the installed version of spaCy
spacy.__version__

# Check GPU information
# !nvidia-smi

'3.5.2'

In [4]:
import os
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner'

In [7]:
# Load the annotated data from a JSON file
df = json.load(open(r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\example_100.json','r', encoding="utf-8"))

# Display the number of items in the dataset
len(df)

# Display the first item in the dataset
df['annotations'][0]

['SOCIEDADES ANÓNIMAS\r\nACEGO\xa0S.A.\r\nEscritura Nº\xa0757 F° 2011 Reg. 553 de fecha 5/7/2023. Federico Gastón GOÑI, nacido el 21/5/1986, DNI 32.257.649, \r\narquitecto, domiciliado en Calle 9, Nro 1521, Piso 2, Depto A, La Plata, Pcia Bs As, y Gabriel Sebastian GRAU, \r\nnacido el 17/6/1984, DNI 30.923.019, empresario, domiciliada en Calle 49 Nro 339, Planta Baja, Depto B, La Plata, \r\nPcia Bs As; ambos argentinos y solteros. Objeto: A) Realización de obras públicas y/o privadas, por contratación \r\ndirecta, licitación pública y/o privada y cualquier sistema vigente y/o a crearse, ejecución de proyecto, dirección, \r\nadministración, construcción, reparación y reformación de todo tipo de obras de ingeniería y arquitectura de \r\ninmuebles, incluyendo las actividades de la albañilería, herrería, electricidad y afines.- B) Compra, venta, \r\nimportación, exportación, y comercialización de todo tipo de materiales, artículos e insumos para la construcción, \r\nincluyendo materiales d

In [8]:
# del cv_data['classes']
df.keys()
data = df['annotations']
indices_to_remove = []
for i,d in enumerate(data):
    try:
        if d is None:
            indices_to_remove.append(i)
    except:
        pass
# print(indices_to_remove)
data = [value for index, value in enumerate(data) if index not in indices_to_remove]
len(data)

64

In [28]:
# Initialize spaCy configuration files by copying from base_config to config.cfg
# !python -m spacy init fill-config /"your_file_path"/base_config.cfg /"your_file_path"/config/config.cfg
!python -m spacy init fill-config C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\config.cfg

[paths]
train = null
dev = null
vectors = "es_core_news_lg"
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "es"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
wid

In [10]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('es')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [11]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

(51, 13)

In [ ]:
r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\example_100.json'

In [12]:
# Open a file to log errors during annotation processing
file = open(r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk(r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk(r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 13/13 [00:00<00:00, 154.40it/s]


In [38]:
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner'

In [39]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [1]:
# !python -m spacy download es_core_news_lg


In [42]:
import spacy
from spacy.training.example import Example

# Download the Spanish model
spacy.cli.download("es_core_news_lg")

# Load the Spanish model
nlp = spacy.load("es_core_news_lg")

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [43]:
!python -m spacy train config.cfg --output C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\ --paths.train C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\train_data.spacy --paths.dev C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\test_data.spacy

[i] Saving to output directory:
C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\ner_files
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     99.14    0.00    0.00    0.00    0.00
  3     200       1893.55   5391.24   62.71   60.66   64.91    0.63
  7     400         19.94    367.04   76.79   78.18   75.44    0.77
 11     600         29.78    199.94   78.46   69.86   89.47    0.78
 15     800         42.94    124.95   83.19   83.93   82.46    0.83
 19    1000        112.12    196.42   70.49   66.15   75.44    0.70
 23    1200        131.54    150.11   79.03   73.13   85.96    0.79
 27    1400         52.57

[2024-01-30 16:08:15,266] [INFO] Set up nlp object from config
[2024-01-30 16:08:15,281] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-01-30 16:08:15,285] [INFO] Created vocabulary
[2024-01-30 16:08:16,561] [INFO] Added vectors: es_core_news_lg
[2024-01-30 16:08:17,930] [INFO] Finished initializing nlp object
[2024-01-30 16:08:22,967] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


# EJECUCCION

In [5]:
text = '''
KEYBOARD S.A.
Escritura Nº 362 de fecha 27/06/2023. Orlando DI FONZO, argentino, empresario, casado en primeras nupcias con 
Amalia Rinaldi, DNI 8.609.671, CUIT Nº 20-08609671-5; y Gonzalo Gastón DI FONZO, argentino, empresario, soltero, 
DNI 33.689.504, CUIT 23-33689504-9, ambos domiciliados en Carmen Puch Nº 1000, de la Localidad y Partido de 
Pilar, Provincia de Buenos Aires: Resuelven que la sociedad se adecúe voluntariamente a las disposiciones del art. 
124 de la ley 19.550 y aprueban todas las estipulaciones del estatuto: 1) Keyboard S.A. 2) Duración: 30 años desde 
inscripción. 3) Objeto: Compra, venta, hipoteca, arrendamiento y toda clase de operaciones con bienes inmuebles. 
Realizar y/o administrar inversiones. Importaciones, exportaciones, comisiones, representaciones y mandatos. 
Operaciones financieras relacionadas con sus actividades principales. 4) Capital: $  1.000.000.-, representado 
por diez mil acciones ordinarias, nominativas no endosables, de v/n $ 100.- cada una y derecho a un voto por 
acción. 5) Dirección y administración: a cargo del directorio compuesto del número de miembros que determine 
la asamblea, entre un mínimo de uno y un máximo de cinco, por tres ejercicios. 6) Prescindencia de Sindicatura. 
7) Suscripción: Orlando DI FONZO y Gonzalo Gastón DI FONZO suscriben la cantidad de cinco mil acciones 
cada uno, lo que representa el 50% del capital social. Directorio: PRESIDENTE: Orlando DI FONZO; y DIRECTOR 
SUPLENTE: Gonzalo Gastón DI FONZO, quienes constituyen domicilio especial en la sede social, fijada en la calle 
Lavalle Nº 643, Piso 5º, Oficina “D”, C.A.B.A. Cierre ejercicio social: 30 de junio de cada año. Autorizado según 
instrumento público Esc. Nº 362 de fecha 27/06/2023 Reg. Nº 1017.
Lucía Agundez - T°: 114 F°: 540 C.P.A.C.F.
'''

In [6]:
# Load a model from a specific path
custom_model_path = r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner\\ner_files\\model-best'
nlp = spacy.load(custom_model_path)

In [7]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)
# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

Orlando DI FONZO   ->>>>   SOCIO
Gonzalo Gastón DI FONZO   ->>>>   SOCIO
Lucía Agundez   ->>>>   ESCRIBANO


In [8]:
doc


KEYBOARD S.A.
Escritura Nº 362 de fecha 27/06/2023. Orlando DI FONZO, argentino, empresario, casado en primeras nupcias con 
Amalia Rinaldi, DNI 8.609.671, CUIT Nº 20-08609671-5; y Gonzalo Gastón DI FONZO, argentino, empresario, soltero, 
DNI 33.689.504, CUIT 23-33689504-9, ambos domiciliados en Carmen Puch Nº 1000, de la Localidad y Partido de 
Pilar, Provincia de Buenos Aires: Resuelven que la sociedad se adecúe voluntariamente a las disposiciones del art. 
124 de la ley 19.550 y aprueban todas las estipulaciones del estatuto: 1) Keyboard S.A. 2) Duración: 30 años desde 
inscripción. 3) Objeto: Compra, venta, hipoteca, arrendamiento y toda clase de operaciones con bienes inmuebles. 
Realizar y/o administrar inversiones. Importaciones, exportaciones, comisiones, representaciones y mandatos. 
Operaciones financieras relacionadas con sus actividades principales. 4) Capital: $  1.000.000.-, representado 
por diez mil acciones ordinarias, nominativas no endosables, de v/n $ 100.- cada un

In [9]:
ents = [(e.text, e.label_) for e in doc.ents if e.label_ not in ('DATE', 'TIME', 'ORDINAL', 'CARDINAL')]
# ents = [(k,v) for (v,k) in ents]
# dict(ents)
# Dictlist(ents)
ents

[('Orlando DI FONZO', 'SOCIO'),
 ('Gonzalo Gastón DI FONZO', 'SOCIO'),
 ('Lucía Agundez', 'ESCRIBANO')]

# get probas

In [29]:
dict(ents)

{'Orlando DI FONZO': 'SOCIO',
 'Gonzalo Gastón DI FONZO': 'SOCIO',
 'Lucía Agundez': 'ESCRIBANO'}

In [31]:
print(text)


KEYBOARD S.A.
Escritura Nº 362 de fecha 27/06/2023. Orlando DI FONZO, argentino, empresario, casado en primeras nupcias con 
Amalia Rinaldi, DNI 8.609.671, CUIT Nº 20-08609671-5; y Gonzalo Gastón DI FONZO, argentino, empresario, soltero, 
DNI 33.689.504, CUIT 23-33689504-9, ambos domiciliados en Carmen Puch Nº 1000, de la Localidad y Partido de 
Pilar, Provincia de Buenos Aires: Resuelven que la sociedad se adecúe voluntariamente a las disposiciones del art. 
124 de la ley 19.550 y aprueban todas las estipulaciones del estatuto: 1) Keyboard S.A. 2) Duración: 30 años desde 
inscripción. 3) Objeto: Compra, venta, hipoteca, arrendamiento y toda clase de operaciones con bienes inmuebles. 
Realizar y/o administrar inversiones. Importaciones, exportaciones, comisiones, representaciones y mandatos. 
Operaciones financieras relacionadas con sus actividades principales. 4) Capital: $  1.000.000.-, representado 
por diez mil acciones ordinarias, nominativas no endosables, de v/n $ 100.- cada un

In [83]:
from collections import defaultdict

defaultdict(ents)

TypeError: first argument must be callable or None

In [95]:
def modelo_base(text,avoid_ls=None):
    avoid_ls = ['DATE', 'TIME', 'ORDINAL', 'CARDINAL']
    doc = nlp(text)
    ents = [(e.text, e.label_) for e in doc.ents if e.label_ not in avoid_ls]
    return ents

In [96]:
modelo_base(text)

[('Orlando DI FONZO', 'SOCIO'),
 ('Gonzalo Gastón DI FONZO', 'SOCIO'),
 ('Lucía Agundez', 'ESCRIBANO')]

In [55]:
df_base = pd.read_pickle('df.pkl')

In [109]:
df_base[['texto']].values.tolist()

363000

In [111]:
for t in df_base[['texto']][1000:1050].values.tolist():
    pass

In [113]:
df_base[['texto']][1000:1050].values.tolist()[0]

['\nPALOKO\xa0S.R.L.\nCUIT 30564526408. Mediante escritura 347 del 29/05/2023, folio 1141 del Registro 284 de la Capital Federal, se \nDESIGNA GERENTE ADICIONAL: por el término de duración de la sociedad a Linet Flavia PERUZZO SCUSSEL, \nargentina, 20/11/1987, DNI 33573596, CUIT 27335735965, comerciante, casada, quien acepta el cargo y constituye \ndomicilio en Av. San Martín 2269, CABA. Continuando en el cargo Ivana Licia SCUSSEL. Autorizado según \ninstrumento público Esc. Nº\xa0347 de fecha 29/05/2023 Reg. Nº\xa0284\nEnrique Fernandez Moujan - Matrícula: 4009 C.E.C.B.A.\n']

In [119]:
resultados_ls = []
for text in df_base[df_base.chapter=='CONTRATOS SOBRE PERSONAS JURÍDICAS'][['texto']][1000:1050].values.tolist():
    resultados_ls.append(modelo_base(text[0]))
    

In [122]:
casos = df_base[df_base.chapter=='CONTRATOS SOBRE PERSONAS JURÍDICAS'][['texto']][1000:1050].values.tolist().copy()
print(casos[0][0])


MHI POWER ARGENTINA S.R.L.
(30-71574699-5) Edicto rectificatorio del publicado el 07/03/2023 N° 12936/23 v. 07/03/2023, según fuera ordenado 
por la Inspección General de Justicia en fecha 05/06/2023. En el Punto ii “léase: El capital social quedó así 
compuesto: Mitsubishi Power Américas Inc., titular de 2.537.770. cuotas sociales, de VN $ 10 por cuota social y 
MHI Power Florida LLC, titular de 316 cuotas sociales de VN $ 10 por cuota social.
Autorizado según instrumento privado Acta de Reunión de Socios de fecha 13/12/2021
Maria Agustina Hagelstrom - T°: 139 F°: 344 C.P.A.C.F.



In [124]:
print(casos[1][0])


MYG GESTION S.R.L.
Por escritura del 13/07/2023 se constituyo la sociedad. Socios: Gonzalo Damián GASPAROTTI, argentino, casado, 
contador público, 8/02/1977, DNI 25.340.650, Vuelta de Obligado 2.371, piso 8, departamento A, CABA, 100 cuotas 
y Mariano OTERO, argentino, casado, contador público, 30/01/1.973, DNI 22.890.449, J. M. de Estrada 3447, 
Localidad de La Lucila, Partido de Vicente López, Provincia de Buenos Aires, 100 cuotas, Plazo: 30 AÑOS, Objeto: la 
prestación de servicios de consultoría, asesoramiento e información a personas físicas y jurídicas, en cuestiones de 
asistencia técnica en materia contable, y fiscal de la empresa, planificación financiera, fortalecimiento de recursos, 
implementación de estrategias comerciales. Diagnóstico de problemática empresarial, estableciendo estrategias 
que permitan mejorar su productividad, liquidez y estructura, contribuyendo a su optimización financiera y todas 
aquellas otras actividades propias de las incumbencias recaídas en la

In [125]:
resultados_ls[1]

[('Gonzalo Damián GASPAROTTI', 'SOCIO'),
 ('Gerardo Daniel Ricoso', 'ESCRIBANO')]

In [120]:
resultados_ls

[[('Maria Agustina Hagelstrom', 'ESCRIBANO')],
 [('Gonzalo Damián GASPAROTTI', 'SOCIO'),
  ('Gerardo Daniel Ricoso', 'ESCRIBANO')],
 [('Fernando Mariano CAROTTA', 'SOCIO'),
  ('Rosa Margarita VILA', 'SOCIO'),
  ('Diego Luaces', 'ESCRIBANO')],
 [('Shengfeng YU', 'SOCIO'),
  ('Jiawei HE', 'SOCIO'),
  ('Long YE', 'SOCIO'),
  ('VERONICA PAOLA ZAZZALI', 'ESCRIBANO')],
 [('Jairo Alonso Moreno Prada', 'SOCIO'),
  ('Tobías Nicolás Molina', 'SOCIO'),
  ('Lucas Emanuel Fortete', 'ESCRIBANO')],
 [('Daniel Alejandro Yazbek', 'SOCIO'),
  ('María Belén Yazbek', 'SOCIO'),
  ('Gastón Ezequiel Scrivano', 'SOCIO')],
 [('Mariela Soledad Lopez', 'SOCIO'),
  ('Verónica Alejandra Monciino', 'SOCIO'),
  ('ana maria valdez', 'ESCRIBANO')],
 [('Marcelo Carlos Mombelli', 'SOCIO'),
  ('Verónica Alejandra Monciino', 'SOCIO'),
  ('ana maria valdez', 'ESCRIBANO')],
 [('Q SERVICE & TECHNOLOGY\xa0S.R.L.', 'SOCIEDAD'),
  ('Juan Facundo del VALLE', 'SOCIO'),
  ('Héctor Florencio GATTI', 'SOCIO'),
  ('Gustavo \nMarcelo 

In [118]:
df_base[df_base.chapter=='CONTRATOS SOBRE PERSONAS JURÍDICAS']

,chapter,escrito,texto
0,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55783/23 v. 19/07/2023,SOCIEDADES ANÓNIMAS\nACEGO S.A.\nEscritura Nº ...
1,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55773/23 v. 19/07/2023,\nAGROQUIM NORTE S.A.\nEsc 52 del 03/07/2023 F...
2,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55779/23 v. 19/07/2023,\n\n\n \nALCARI S.A.\nNúmero Correlativo 1.707...
3,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55528/23 v. 19/07/2023,\nB2FI S.A.\n1) 17/07/2023. 2) Francisco LEVIS...
4,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55539/23 v. 19/07/2023,\nCMIX ARGENTINA S.A.\nCUIT: 30-71088679-9.Com...
...,...,...,...
362805,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55671/23 v. 19/07/2023,\nSYSTRADER S.R.L.\nCUIT 30-71425080-5. 1) Por...
362806,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55748/23 v. 19/07/2023,\nTHE FIRST LAW S.R.L.\n1) 17/7/23. 2) María C...
362807,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55512/23 v. 19/07/2023,\nTHE LEAD BANG S.R.L.\nComplemento aviso de f...
362808,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55527/23 v. 19/07/2023,\nTHE TOY STORE S.R.L.\n1) 14/07/2023. 2) Sant...


In [115]:
resultados_ls

[[('Enrique Fernandez Moujan', 'ESCRIBANO')],
 [('Miguel Carlos Remmer', 'SOCIO')],
 [('Anibal Enrique ALFONZO', 'SOCIO')],
 [('MARIA LAURA ANSELMI', 'ESCRIBANO')],
 [('María Dolores Garcia', 'ESCRIBANO')],
 [('Maria Lucia Quinteiro', 'ESCRIBANO')],
 [('Adriana Amelia Cavallo -', 'SOCIO')],
 [('Adriana Amelia Cavallo -', 'SOCIO')],
 [('Adriana Amelia Cavallo -', 'SOCIO')],
 [('Milagros María Pelly', 'ESCRIBANO')],
 [('Miguel Carlos Remmer', 'SOCIO')],
 [('María Fernanda Mierez', 'ESCRIBANO')],
 [('María Cristina Lopez Seoane', 'SOCIO')],
 [('MARINA GRIGLIONE HESS - T', 'SOCIO')],
 [('MERCEDES MARIA VERON', 'ESCRIBANO')],
 [('Gabriela Eliana Blanco', 'SOCIO')],
 [('Jose Luis Alvarez', 'ESCRIBANO')],
 [('Gabriela Fernanda Casas', 'SOCIO')],
 [('Joaquín Estanislao Martínez -', 'SOCIO')],
 [('Nicolas Santurio', 'ESCRIBANO')],
 [('Sr. Hernán José Marsili', 'SOCIO'), ('LUCIANA PADRONI', 'ESCRIBANO')],
 [('Fernando Enrique Freire -', 'SOCIO')],
 [],
 [('Nicolás LINO cedió 30.000', 'SOCIO'),
 